# EDA
## cleaning data: test에 없는 정보 삭제

In [10]:
df = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pickle

df = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

c = df['DepartmentDescription'].isin(test['DepartmentDescription'])
c2 = df['Upc'].isin(test['Upc'])
c3 = df['FinelineNumber'].isin(test['FinelineNumber'])

# test에 없는 데이터 삭제
df = df[c&c2&c3]

# fill na: 여기서는 카테고리값 -> 임의로 "0.11 카테고리"로 지정
df['FinelineNumber'] = df['FinelineNumber'].fillna(0.11)
test['FinelineNumber'] = test['FinelineNumber'].fillna(0.11)

df['Upc'] = df['Upc'].fillna(0.11)
test['Upc'] = test['Upc'].fillna(0.11)

df['DepartmentDescription']= df['DepartmentDescription'].fillna(0.11)
test['DepartmentDescription'] = test['DepartmentDescription'].fillna(0.11)

In [22]:
#original
## test 중 df에 없는것
print(len(np.where(~np.isin(test['FinelineNumber'].unique(), df['FinelineNumber'].unique()))[0]))
## df 중 test에 없는것
print(len(np.where(~np.isin(df['FinelineNumber'].unique(), test['FinelineNumber'].unique()))[0]))

159
151


In [24]:
#original
## test 중 df에 없는것
## df 중 test에 없는것
print(len(np.where(~np.isin(test['FinelineNumber'].unique(), df['FinelineNumber'].unique()))[0]))
print(len(np.where(~np.isin(df['FinelineNumber'].unique(), test['FinelineNumber'].unique()))[0]))

322
0


In [20]:
#original
## test 중 df에 없는 것
print(len(np.where(~np.isin(test['Upc'].unique(), df['Upc'].unique()))[0]))
## df 중 test에 없는 것
print(len(np.where(~np.isin(df['Upc'].unique(), test['Upc'].unique()))[0]))

26980
26547


In [25]:
#original
## test 중 df에 없는 것
print(len(np.where(~np.isin(test['Upc'].unique(), df['Upc'].unique()))[0]))
## df 중 test에 없는 것
print(len(np.where(~np.isin(df['Upc'].unique(), test['Upc'].unique()))[0]))

26980
0


# Join DataFrame

## train

- join extra info

In [ ]:
import pandas as pd
# label data
temp = df[['VisitNumber', 'TripType']].drop_duplicates('VisitNumber')
temp = temp.set_index('VisitNumber')
temp = pd.read_pickle('temp_merge.pkl')

In [2]:
frame_train_fine_pca = pd.read_pickle('pca_fine_low.pkl')
frame_train_fine_kmeans = pd.read_pickle('frame_fine_kmeans_uni.pkl')
frame_dept = pd.read_pickle('frame_pca_dept.pkl')

In [3]:
frame_join = frame_dept\
.merge(frame_train_fine_pca, right_index=True, left_index=True)\
.merge(frame_train_fine_kmeans, right_index=True, left_index=True)

In [4]:
# TripType 추가
frame_join = frame_join\
.merge(temp, right_index=True, left_on='VisitNumber')

- join basic info

In [5]:
frame_day = pd.read_pickle('frame_feature_basic.pkl')
frame_join = frame_join\
.merge(frame_day, left_index=True, right_index=True)

- 최종 데이터

In [6]:
frame_join.to_pickle('join_second_basic.pkl')

In [7]:
len(frame_join.columns), len(frame_join)

(4195, 93213)

## test

- join extra info

In [8]:
import pandas as pd
frame_test_fine_pca = pd.read_pickle('pca_fine_low_test.pkl')
frame_test_fine_kmeans = pd.read_pickle('frame_fine_kmeans_uni_test.pkl')
frame_dept_test = pd.read_pickle('frame_pca_dept_test.pkl')

In [9]:
frame_join_test = frame_dept_test\
.merge(frame_test_fine_pca, right_index=True, left_index=True)\
.merge(frame_test_fine_kmeans, right_index=True, left_index=True)

- join basic info

In [10]:
frame_day_test = pd.read_pickle('frame_feature_basic_test.pkl')

frame_join_test = frame_join_test\
.merge(frame_day_test, left_index=True, right_index=True)\

- 최종 데이터

In [11]:
frame_join_test.to_pickle('join_second_basic_test.pkl')

In [12]:
len(frame_join_test.columns), len(frame_join_test)

(4194, 95674)

# 추가적인 Join

In [1]:
import pandas as pd
frame_join = pd.read_pickle('join_second_basic_upc.pkl')
#frame_join_test = pd.read_pickle('join_second_basic_upc_test.pkl')

In [2]:
extra = pd.read_pickle('frame_upc_num.pkl')
#extra_test = pd.read_pickle('frame_upc_num_test.pkl')

In [3]:
frame_join = frame_join.merge(extra, right_index=True, left_index=True)
#frame_join_test = frame_join_test.merge(extra_test, right_index=True, left_index=True)

In [4]:
frame_join.to_pickle('join_all_upc_num.pkl')
#frame_join_test.to_pickle('join_all_upc_num_test.pkl')